# 1. Ở version trước Controller, routes, server.js đã có 
- Ngoài ra chưa có service??? 

Được rồi, dưới đây là bảng tóm tắt chi tiết về **Routes**, **Controllers**, và **Services** trong ứng dụng web, được giải thích đơn giản cho học sinh cấp 2:

---

### **Bảng tóm tắt**

| **Thành phần**     | **Ví dụ đời thường**                                       | **Vai trò trong ứng dụng web**                                                          |
|--------------------|------------------------------------------------------------|-----------------------------------------------------------------------------------------|
| **Routes (Đường dẫn)**       | **Cửa ra vào của nhà hàng**                                  | - Định nghĩa các địa chỉ (URL) mà người dùng truy cập.<br>- Chuyển yêu cầu của người dùng đến **Controller** phù hợp. |
| **Controller (Người phục vụ)** | **Nhân viên phục vụ nhận gọi món từ khách hàng**              | - Nhận yêu cầu từ người dùng qua **Routes**.<br>- Hiểu người dùng muốn gì và quyết định cần làm gì.<br>- Gọi **Service** để thực hiện công việc chính.<br>- Trả kết quả lại cho người dùng. |
| **Service (Nhà bếp)**        | **Đầu bếp trong nhà bếp nấu món ăn theo yêu cầu**             | - Thực hiện công việc chính (xử lý dữ liệu, tính toán).<br>- Tương tác với cơ sở dữ liệu hoặc dịch vụ bên ngoài.<br>- Trả kết quả về cho **Controller** sau khi hoàn thành. |


Let me explain the difference between Controllers and Services in a backend architecture, and suggest how we can improve the current codebase.

Currently, our controllers are handling both request/response logic AND business logic. This violates the Single Responsibility Principle. Here's how we should restructure it:

### Controllers should:
- Handle HTTP requests/responses
- Validate input data
- Call appropriate services
- Return responses

### Services should:
- Contain business logic
- Handle data processing
- Interact with external services (like OpenAI)
- Handle database operations

Let's refactor one example (the Learning Card feature) to demonstrate:

1. First, create a service:

```javascript:backend/src/services/learningCardService.js
const OpenAI = require('openai');

const openai = new OpenAI({
    apiKey: process.env.OPENAI_API_KEY
});

class LearningCardService {
    constructor() {
        this.LEARNING_CARD_PROMPT = `You are an English exercise content expert...`; // Move prompt here
    }

    async generateCards(lessons) {
        const allResults = [];
        
        for (const lesson of lessons) {
            const lessonPrompt = this._formatLessonPrompt(lesson);
            const response = await this._callOpenAI(lessonPrompt);
            const parsedResults = this._parseAndValidateResponse(response);
            allResults.push(...parsedResults);
        }
        
        return allResults;
    }

    _formatLessonPrompt(lesson) {
        return JSON.stringify({
            structure_en: lesson.structure,
            main_phrase: lesson["main phrase"],
            optional_phrase_1: lesson["optional phrase 1"],
            optional_phrase_2: lesson["optional phrase 2"]
        }, null, 2);
    }

    async _callOpenAI(prompt) {
        return await openai.chat.completions.create({
            model: 'gpt-4o-mini',
            messages: [
                { role: 'system', content: this.LEARNING_CARD_PROMPT },
                { role: 'user', content: prompt }
            ],
            max_tokens: 3000,
            temperature: 0
        });
    }

    _parseAndValidateResponse(response) {
        const content = response.choices[0].message.content;
        const cleanedContent = content.trim().replace(/```json|```/g, '');
        const results = JSON.parse(cleanedContent);
        
        if (!Array.isArray(results) || results.length !== 4) {
            throw new Error('Invalid response format');
        }
        
        return results;
    }
}

module.exports = new LearningCardService();
```

2. Then simplify the controller:

```javascript:backend/src/controllers/generateLearningCardController.js
const learningCardService = require('../services/learningCardService');

exports.generateLearningCard = async (req, res) => {
    try {
        // Input validation
        if (!req.body.lessons || !Array.isArray(req.body.lessons)) {
            return res.status(400).json({ error: 'Invalid input format' });
        }

        // Call service
        const results = await learningCardService.generateCards(req.body.lessons);
        
        // Return response
        res.json(results);
        
    } catch (error) {
        console.error('Error in generateLearningCard:', error);
        res.status(500).json({ 
            error: error.message,
            details: error.stack
        });
    }
};
```

Benefits of this separation:

1. **Single Responsibility**: Each component has a clear, focused role
2. **Testability**: Services can be tested independently of HTTP logic
3. **Reusability**: Services can be used by different controllers
4. **Maintainability**: Easier to modify business logic without touching request handling
5. **Error Handling**: Clear separation between HTTP errors and business logic errors

Looking at the current code:

```1:151:backend/src/controllers/generateLearningCardController.js
const OpenAI = require('openai');

const openai = new OpenAI({
    apiKey: process.env.OPENAI_API_KEY
});

const LEARNING_CARD_PROMPT = `You are an English exercise content expert. Given structure_en, main_phrase, and up to two optional_phrases, create a JSON array with:

- sentence_en: English sentence
- sentence_vi: Vietnamese translation 
- ipa: IPA pronunciation

**Response Format:** Output only in JSON format with no extra characters (such as \`\`\`Json).

**Example Input:**
{
    "structure_en": "Our team of _____ specialists is here to help",
    "main_phrase": "healthcare",
    "optional_phrase_1": "financial", 
    "optional_phrase_2": "legal"
}

Expected Output:
[
    {
        "sentence_en": "Our team of _____ specialists is here to help",
        "sentence_vi": "Đội ngũ chuyên gia _____ của chúng tôi sẵn sàng hỗ trợ",
        "ipa": "/ˈaʊər tiːm əv _____ ˈspɛʃəlɪsts ɪz hɪər tuː hɛlp/"
    },
    {
        "sentence_en": "healthcare specialists",
        "sentence_vi": "chuyên gia y tế",
        "ipa": "/ˈhɛlθˌkɛr ˈspɛʃəlɪsts/"
    },
    {
        "sentence_en": "financial specialists", 
        "sentence_vi": "chuyên gia tài chính",
        "ipa": "/faɪˈnænʃəl ˈspɛʃəlɪsts/"
    },
    {
        "sentence_en": "legal specialists",
        "sentence_vi": "chuyên gia pháp lý", 
        "ipa": "/ˈliːɡəl ˈspɛʃəlɪsts/"
    }
]`;
exports.generateLearningCard = async (req, res) => {
    try {
        // Log request data
        console.log('Received request for learning cards:', req.body);
        
        const { lessons } = req.body;
        const allResults = [];
        
        for (const lesson of lessons) {
            // Log mỗi lesson được xử lý
            console.log('Processing lesson:', lesson);
            
            const lessonPrompt = JSON.stringify({
                structure_en: lesson.structure,
                main_phrase: lesson["main phrase"],
                optional_phrase_1: lesson["optional phrase 1"],
                optional_phrase_2: lesson["optional phrase 2"]
            }, null, 2);
            
            // Log prompt trước khi gửi tới OpenAI
            console.log('Sending prompt to OpenAI:', lessonPrompt);
            
            const response = await openai.chat.completions.create({
                model: 'gpt-4o-mini',
                messages: [
                    { role: 'system', content: LEARNING_CARD_PROMPT },
                    { role: 'user', content: lessonPrompt }
                ],
                max_tokens: 3000,
                temperature: 0
            });
            
            try {
                const content = response.choices[0].message.content;
                // Log raw response
                console.log('Raw OpenAI response:', content);
                
                const cleanedContent = content.trim().replace(/```json|```/g, '');
                // Log cleaned content
                console.log('Cleaned content:', cleanedContent);
                const lessonResults = JSON.parse(cleanedContent);
                
                // Add processing for IPA
                lessonResults.forEach(result => {
                    result.ipa = result.ipa.replace(/[_]+/g, '');
                });
                
                // Log parsed results
                console.log('Parsed results:', lessonResults);
                
                if (!Array.isArray(lessonResults)) {
                    console.error('Response is not an array:', lessonResults);
                    throw new Error('Response must be an array');
                }

                if (lessonResults.length !== 4) {
                    console.error('Invalid number of results:', lessonResults.length);
                    throw new Error('Expected exactly 4 results');
                }

                const validResults = lessonResults.filter(result => {
                    const isValid = result.sentence_en && 
                                  result.sentence_vi && 
                                  result.ipa;
                    
                    if (!isValid) {
                        console.error('Invalid result structure:', result);
                    }
                    return isValid;
                });

                if (validResults.length === 4) {
                    allResults.push(...validResults);
                } else {
                    throw new Error(`Expected 4 valid results, got ${validResults.length}`);
                }
            } catch (parseError) {
                // Log detailed parse error
                console.error('Parse error details:', {
                    error: parseError.message,
                    stack: parseError.stack,
                    content: response.choices[0].message.content
                });
                throw parseError;
            }
        }
        
        // Log final results
        console.log('Successfully generated cards:', allResults.length);
        res.json(allResults);
        
    } catch (error) {
        // Log error với stack trace
        console.error('Error in generateLearningCard:', {
            message: error.message,
            stack: error.stack
        });
        
        res.status(500).json({ 
            error: error.message,
            details: error.stack
        });
    }
}
```


We can see it's mixing concerns by handling both HTTP and business logic. The refactored version separates these responsibilities clearly.

Would you like me to show how to refactor the other controllers in a similar way?